In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/drive/MyDrive/NLP with Disaster Tweets/train.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df_disaster = df[df['target'] == 1]
df_disaster

In [ ]:
m,n = df.shape
rate = 0.8
X_train = df.iloc[:round(rate*m),-2]
y_train = df.iloc[:round(rate*m),-1].values
X_train.head(), y_train

In [ ]:
X_val = df.iloc[round(rate*m):,-2]
y_val = df.iloc[round(rate*m):,-1].values
X_val.head(), y_val[:5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vocabulary = vect.fit_transform(X_train)
words = vect.get_feature_names()
len(words)

In [ ]:
import nltk
nltk.download('words')
nltk.download('stopwords')

In [ ]:
# Loại bỏ các stopwords
import nltk
from nltk.corpus import stopwords

stops = list(stopwords.words('english'))
for i in range(0,len(stops)):
  if stops[i] in words:
    words.remove(stops[i])

len(words)

In [ ]:
# Loại bỏ các từ vô nghĩa
meaningful = list(nltk.corpus.words.words())
j = 0
while j < len(words):
  if words[j] not in meaningful:
    words.remove(words[j])
  else:
    j += 1
len(words)

In [ ]:
words[:5]

In [ ]:
# test voi X_train[0]
X_train[0]

In [ ]:
import re
l_doc = re.sub(r"[^a-zA-Z0-9]", " ", X_train[0].lower()).split()
l_doc

In [ ]:
import numpy as np
def bag_of_word(sentence):
  vector = np.zeros(len(words))
  for n in range(0,len(words)):
    count = 0
    for word in sentence:
      if words[n]== word:
        count = count + 1
        # print(word, count)
    vector[n] = count 
  return vector

bag_of_word(l_doc)
# print('vector: ', bag_of_word(l_doc)[1500:2000])

In [ ]:
import numpy as np
import re

i = 0
X_vector = []
for i in range(0,len(X_train)):
  l_doc = re.sub(r"[^a-zA-Z0-9]", " ", X_train[i].lower()).split()
  vector = bag_of_word(l_doc)
  X_vector.append(vector)

X_vector

In [ ]:
len(X_vector)

In [ ]:
from sklearn import svm
classifier = svm.SVC(C=10,kernel="rbf")
classifier.fit(X_vector,y_train)


In [ ]:
i = 0
X_vector_test = []
for i in range(round(rate*m),len(df)):
  l_doc = re.sub(r"[^a-zA-Z0-9]", " ", X_val[i].lower()).split()
  vector = bag_of_word(l_doc)
  X_vector_test.append(vector)

len(X_vector_test)

In [ ]:
y_predict = classifier.predict(X_vector_test)

from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_val, y_predict))
print("Precision:",metrics.precision_score(y_val, y_predict))
print("Recall:",metrics.recall_score(y_val, y_predict))
print("F1_Score:",metrics.f1_score(y_val, y_predict))

**Cải thiện mô hình bằng phương pháp PCA**

In [ ]:
# Improve with PCA
from sklearn.decomposition import PCA

pca = PCA(0.9)
X_vector_PCA = pca.fit_transform(X_vector)

n_components = pca.n_components_

PCA_test = PCA(n_components)

n_components, PCA_test, pca

In [ ]:
X_vector_PCA.shape

In [ ]:
from sklearn import svm
classifier = svm.SVC(C=10,kernel="rbf")
classifier.fit(X_vector_PCA,y_train)
i = 0
X_vector_test = []
for i in range(round(rate*m),len(df)):
  l_doc = re.sub(r"[^a-zA-Z0-9]", " ", X_val[i].lower()).split()
  vector = bag_of_word(l_doc)
  X_vector_test.append(vector)

In [ ]:
PCA_test = PCA(n_components)
X_vector_test = np.array(X_vector_test)
X_test_PCA = PCA_test.fit_transform(X_vector_test)
X_test_PCA.shape, y_val.shape

In [ ]:
y_predict = classifier.predict(X_test_PCA)

from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_val, y_predict))
print("Precision:",metrics.precision_score(y_val, y_predict))
print("Recall:",metrics.recall_score(y_val, y_predict))
print("F1_Score:",metrics.f1_score(y_val, y_predict))

Ta nhận thấy kết quả thu được không như mong đợi vì:
1.   Với số chiều là 761 là quá nhỏ, phần thông tin bị lượt bỏ quá nhiều so với khoảng 950 chiều tương đương 90% thông tin.
2.   Hạn chế trong phương thức PCA gặp lỗi nếu số chiều feature chỉ có thể nằm trong khoảng 0 đến min(sample). Nghe hơi vô lí nhưng tôi cần xem xét lại. Để giải quyết vấn đề này chúng ta cần số mẫu thử (test) lớn hơn. Hy vọng có thể cải thiện được mô hình 

